In [1]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import gtfstk as gt
import shapely.geometry as sg
import shapely.ops as so
import folium as fl

sys.path.append('../')

import make_gtfs as mg


DATA_DIR = Path('../data')

%load_ext autoreload
%autoreload 2


In [2]:
gt.__version__

'9.2.2'

In [3]:
import shapely.wkt as sw
import shapely.ops as so

#s = sg.shape(pfeed.shapes['features'][0]['geometry'])
s = sw.loads("LINESTRING(0 0, 0.2 0.1, 0.3 0.6, 0.4 0.1, 1 1)")
#s = sg.LineString([(x, y) for x, y in reversed(s.coords)])

eps = 10e-3
b0 = s.buffer(eps, cap_style=3)
b1 = s.buffer(0.05, cap_style=2)
diff = b1.difference(b0)
left, right = list(so.polygonize(diff))


In [4]:
from matplotlib import pyplot
from shapely.geometry import LineString
from descartes.patch import PolygonPatch

fig = pyplot.figure(1, figsize=(10, 4), dpi=180)


ax = fig.add_subplot(121)
patch2a = PolygonPatch(left, facecolor='#cccccc', edgecolor='#999999')
ax.add_patch(patch2a)
patch2b = PolygonPatch(right, facecolor='#99ccff', edgecolor='#6699cc')
ax.add_patch(patch2b)

ax.set_xlim(-1, 2)
ax.set_ylim(-1, 2)

pyplot.show()

<Figure size 1800x720 with 1 Axes>

In [5]:
pfeed = mg.ProtoFeed(DATA_DIR/'auckland')
pfeed.shapes_extra

{'shp1': 2, 'shp2': 2}

In [6]:
# Visually test geometrize_stops()

# Compute
geo_stops = gt.geometrize_stops(pfeed.stops, use_utm=True)
linestring_by_shape = mg.build_geometry_by_shape(pfeed, use_utm=True)
linestring = linestring_by_shape['shp2']
stops_by_side = {
    side: mg.get_nearby_stops(geo_stops, linestring, side).to_crs(gt.WGS84)
    for side in ['left', 'right', 'both']
}

# Plot
collection_by_side = {side: json.loads(stops.to_json()) 
  for side, stops in stops_by_side.items()}
center = list(stops_by_side['left'].geometry.iat[0].coords[0])[::-1]
my_map = fl.Map(location=center, zoom_start=13, tiles='cartodbpositron')

# Path
f = pfeed.shapes['features'][1]
prop = f['properties']
prop['color'] = 'black'
path = fl.GeoJson(f,
    name=prop['shape_id'],
    style_function=lambda x: {
      'color': x['properties']['color']},
)
path.add_to(my_map)

# Stops
color_by_side = {
    'left': 'blue',
    'right': 'red',
    'both': 'gray',
}
radius_by_side = {
    'left': 5,
    'right': 5,
    'both': 8,
}
for side in stops_by_side:
    color = color_by_side[side]
    radius = radius_by_side[side]
    for f in collection_by_side[side]['features']:
        prop = f['properties']
        if f['geometry']['type'] == 'Point':
            lon, lat = f['geometry']['coordinates']
            fl.CircleMarker(
                location=[lat, lon],
                radius=radius,
                fill=False,
                color=color,
                weight=2,
            ).add_to(my_map)

my_map

In [7]:
feed = mg.build_feed_from_pfeed(pfeed)


In [8]:
prefix = 't-rB-weekday_peak_1-07:00:00'
t = (
    feed.trips
    .loc[lambda x: x.trip_id.str.startswith(prefix)]
)
tids = [t.trip_id.iat[0], t.trip_id.iat[-1]]
feed.map_trips(tids)
    

{'Africa/Abidjan': 'right',
 'Africa/Accra': 'right',
 'Africa/Addis_Ababa': 'right',
 'Africa/Algiers': 'right',
 'Africa/Asmara': 'right',
 'Africa/Bamako': 'right',
 'Africa/Bangui': 'right',
 'Africa/Banjul': 'right',
 'Africa/Bissau': 'right',
 'Africa/Blantyre': 'left',
 'Africa/Brazzaville': 'right',
 'Africa/Bujumbura': 'right',
 'Africa/Cairo': 'right',
 'Africa/Casablanca': 'right',
 'Africa/Ceuta': 'right',
 'Africa/Conakry': 'right',
 'Africa/Dakar': 'right',
 'Africa/Dar_es_Salaam': 'right',
 'Africa/Djibouti': 'right',
 'Africa/Douala': 'right',
 'Africa/El_Aaiun': 'right',
 'Africa/Freetown': 'right',
 'Africa/Gaborone': 'left',
 'Africa/Harare': 'left',
 'Africa/Johannesburg': 'left',
 'Africa/Juba': 'right',
 'Africa/Kampala': 'left',
 'Africa/Khartoum': 'right',
 'Africa/Kigali': 'right',
 'Africa/Kinshasa': 'right',
 'Africa/Lagos': 'right',
 'Africa/Libreville': 'right',
 'Africa/Lome': 'right',
 'Africa/Luanda': 'right',
 'Africa/Lubumbashi': 'right',
 'Africa/Lusa

In [19]:
pfeed.

['Antigua and Barbuda',
 'Australia',
 'Bahamas',
 'Bangladesh',
 'Barbados',
 'Bhutan',
 'Botswana',
 'Brunei',
 'Cyprus',
 'Dominica',
 'East Timor',
 'Fiji',
 'Grenada',
 'Guyana',
 'Hong Kong',
 'India',
 'Indonesia',
 'Ireland',
 'Jamaica',
 'Japan',
 'Kenya',
 'Kiribati',
 'Lesotho',
 'Macau',
 'Malawi',
 'Malaysia',
 'Maldives',
 'Malta',
 'Mauritius',
 'Mozambique',
 'Namibia',
 'Nauru',
 'Nepal',
 'New Zealand',
 'Niue',
 'Northern Cyprus',
 'Pakistan',
 'Papua New Guinea',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Samoa',
 'Seychelles',
 'Singapore',
 'Solomon Islands',
 'South Africa',
 'Sri Lanka',
 'Suriname',
 'Swaziland',
 'Tanzania',
 'Thailand',
 'Tonga',
 'Trinidad and Tobago',
 'Tuvalu',
 'Uganda',
 'United Kingdom',
 'Zambia',
 'Zimbabwe']

In [11]:
from json2html import json2html as jh
import html


d = {'stop_name': "Ponsonby Rd & K'Rd"}
dd = {}
for k, v in d.items():
    try:
        vv = html.escape(v)
    except AttributeError:
        vv = v
    dd[k] = vv

jh.convert(d), jh.convert(dd)


('<table border="1"><tr><th>stop_name</th><td>Ponsonby Rd &amp; K\'Rd</td></tr></table>',
 '<table border="1"><tr><th>stop_name</th><td>Ponsonby Rd &amp;amp; K&amp;#x27;Rd</td></tr></table>')